
#1.) En la terminal: pip install awscli

#2.) aws configure

#https://boto3.amazonaws.com/v1/documentation/api/1.9.42/guide/quickstart.html

#------------------------------------------------#

#import pdf2image

#usa poppler instalarlo en el path

#http://blog.alivate.com.au/poppler-windows/

Anadir al sistema: 

#https://www.thetopsites.net/article/53960829.shtml

__1. LECTURA LISTA DE DOCUMENTOS__

In [437]:
import glob

docs_pdf =  glob.glob('../Info para entregar/*.pdf')  #glob.glob("i/*.pdf")

In [438]:
docs_pdf[24]

'../Info para entregar\\Doc31.pdf'

In [439]:
#guardar el response
import pickle
f = open("ejemplos_estados_financieros/nombresdoc.pkl","wb")
pickle.dump(docs_pdf,f)
f.close()

__2. PASAR A JGE  - BITS__

In [3]:
from pdf2image import convert_from_bytes

#---------------------------------------------#
#Función: READ PDF TO BYTES
#input: Lista de ubicación de pdf
#output: Read pdf.
#---------------------------------------------#

def pdf_to_bytes(Lista_pdf):
    list_pdfBytes = []
    for pdf in range(len(Lista_pdf)):
        imagesBytes =[]
        images = convert_from_bytes(open(Lista_pdf[pdf], 'rb').read())
        for ii in range(len(images)):
            documentName = 'ejemplos_estados_financieros/temp_file.png'
            #------------------------------#
            #llamar la funcion de limpieza de imagen
            #------------------------------#
            images[ii].save(documentName)
            # Read document content
            with open(documentName, 'rb') as document:
                imBytes = bytearray(document.read())
            #concatenar
            imagesBytes.append(imBytes)
        #concatenasr pdf to bytes
        list_pdfBytes.append(imagesBytes)
    #return
    return list_pdfBytes


In [19]:
%%time

PDFBytes = pdf_to_bytes(Lista_pdf=docs_pdf)

Wall time: 1min 51s


In [27]:
##prinit valores y verificar q eta bien:
print(len(PDFBytes))
print(len(docs_pdf))
print(len(PDFBytes[42]))

50
50
3


In [30]:
#PDFBytes[42][0]

__3. ENVIAR A AWS__

In [28]:
import boto3 
from botocore.config import Config

my_config = Config(
    region_name = 'us-west-2',
    signature_version = 'v4',
    retries = {
        'max_attempts': 10,
        'mode': 'standard'
    }
)


s3_connection = boto3.resource('s3')
# Amazon Textract client
textract = boto3.client('textract', config=my_config)


In [31]:
# Call Amazon Textract

def Read_pdf_AWS(PDF_Bytes):
    responses_pdf = []
    for pdf in range(len(PDF_Bytes)):
        response_page = []
        for page_pdf in range(len(PDF_Bytes[pdf])):
            response = textract.analyze_document(Document={'Bytes': PDF_Bytes[pdf][page_pdf]}, FeatureTypes=["TABLES","FORMS"])
            response_page.append(response)
        responses_pdf.append(response_page)
    ###retorno
    return responses_pdf


In [85]:
%%time 

final_responses = Read_pdf_AWS(PDF_Bytes=PDFBytes)

Wall time: 8min 39s


In [87]:
len(final_responses)

50

In [89]:
#guardar el response
import pickle
f = open("ejemplos_estados_financieros/primeros_bbva5.pkl","wb")
pickle.dump(final_responses,f)
f.close()

In [38]:
len(final_responses[0])

2

CARGAR EL RESPONSE

In [420]:
import pickle

final_responses = pickle.load( open("ejemplos_estados_financieros/primeros_bbva5.pkl", "rb" ) )

In [421]:
len(final_responses)

50

__EXTRAER EL TEXTO__

In [422]:

#lee segun las lineas

def PDF_Extraer_Texto(PDF_RESPONSE):
    ## recorre pdf ###
    PDF_texto = []
    for pdf in range(len(PDF_RESPONSE)):
        PAGE_texto = []
        for page  in range(len(PDF_RESPONSE[pdf])):
            ## recorre paginas ##
            response = PDF_RESPONSE[pdf][page]
            text_ini = ''
            for ii in range(len(response["Blocks"])):
                if response["Blocks"][ii]["BlockType"] =="LINE":
                    text_ini = text_ini + ' ' +  response["Blocks"][ii]["Text"] + ' '  
                
            #----------------#
            #----------------#
            PAGE_texto.append(text_ini)
        PDF_texto.append(PAGE_texto)
    return PDF_texto
    

In [423]:
%%time
TEXTO_PDFS = PDF_Extraer_Texto(final_responses)

Wall time: 28.4 ms


In [110]:
#print(TEXTO_PDFS[21][0])

__EXTRAER TABLA__

In [424]:
from trp import Document
import pandas as pd
import numpy as np

#---------------------------------------------#
def tipe_documento(response):
    #optimiza la lectura, detecta cuales son LINE
    indice_line = []
    for ii in range(len(response["Blocks"])):
         indice_line.append(response["Blocks"][ii]["BlockType"])
    return len(list(set(indice_line)))

#---------------------------------------------#
def Retorna_tabla(response):
    doc = Document(response)
    Final_table = []
    cell_text = []
    for page in doc.pages:
         # Print tables
        for table in page.tables:
            #definicio de filas
            rows = len(table.rows)
            #definicion de columnas
            cols = len(table.rows[0].cells)
            #creación de tabla
            tabla_doc = pd.DataFrame(index=np.arange(cols), columns=np.arange(rows))
            #lectura de tabla
            for r, row in enumerate(table.rows):
                for c, cell in enumerate(row.cells):
                    tabla_doc[r][c] = cell.text
                    cell_text.append(cell_text)
            #Concatenar las tablas del doc
            Final_table.append(tabla_doc.transpose())
    return Final_table, cell_text
#---------------------------------------------#
def TABLAS_pdf(PDF_RESPONSE):
    ## recorre pdf ###
    PDF_tablas = []
    for pdf in range(len(PDF_RESPONSE)):
        #print('DOCUMENTO :' + str(pdf) + '>' + docs_pdf[pdf])
        PAGE_tabla = []
        for page  in range(len(PDF_RESPONSE[pdf])):
            #print('PAGINA: ' + str(page))
            ## recorre paginas ##
            response_page = PDF_RESPONSE[pdf][page]
            long_page = tipe_documento(response_page)
            if  long_page > 1:
                tabla_pdf, text_cell = Retorna_tabla(response=response_page)
                PAGE_tabla.append(tabla_pdf)
        PDF_tablas.append(PAGE_tabla)
    return PDF_tablas



In [425]:
%%time

PDF_TABLAS = TABLAS_pdf(final_responses)


Wall time: 4.51 s


In [239]:
pdf = 41
page = 1
print(len(final_responses[pdf]))
print(len(PDF_TABLAS[41]))
print(len(PDF_TABLAS[41][0]))
print(len(PDF_TABLAS[41][1]))

#PDF_TABLAS[pdf][page][0]

3
2
2
1


__4. ELIMINAR INFORMACION DE TEXTO__

In [414]:
import numpy as np
import re

def TEXTO_FINAL(PDF_TABLAS, TEXTO_PDFS):
    TEXTO_DOCS = []
    for pdf in range(len(PDF_TABLAS)):
        TEXTO_PAGES = []
        #print('___document___ ' + str(pdf))
        for page in range(len(PDF_TABLAS[pdf])):
            TEXTO_FINAL = TEXTO_PDFS[pdf][page]
            texto_tabla  = []
            for table in range(len(PDF_TABLAS[pdf][page])):
                lista_text = PDF_TABLAS[pdf][page][table].values
                text_cell = list(set(np.concatenate( lista_text, axis=0 ).tolist()))
                if '' in text_cell:
                    text_cell.remove('')
                texto_tabla.append(text_cell)
            #print('page___ ' + str(page))
            texto_tabla = list(set(np.concatenate(texto_tabla)))
            regex_text = '|'.join(texto_tabla)
            regex_text = re.sub('\(|\)','', regex_text)
            regex_text = re.sub('\+','', regex_text)
            regex_text = re.sub('\-','', regex_text)
            regex_text = re.sub('\[|\]','', regex_text)
            regex_text = re.sub('\$','', regex_text)
            regex_text = re.sub('\.','', regex_text)
            regex_text = re.sub('\| ','', regex_text)
    
            
            TEXTO_FINAL= re.sub('\(|\)','', TEXTO_FINAL)
            TEXTO_FINAL= re.sub('\+','', TEXTO_FINAL)
            TEXTO_FINAL= re.sub('\-','', TEXTO_FINAL)
            TEXTO_FINAL= re.sub('\[|\]','', TEXTO_FINAL)
            TEXTO_FINAL= re.sub('\| ','', TEXTO_FINAL)
            TEXTO_FINAL= re.sub('\$','', TEXTO_FINAL)
            TEXTO_FINAL= re.sub('\.','', TEXTO_FINAL)

            TEXTO_FINAL = re.sub(regex_text, '', TEXTO_FINAL)
            TEXTO_FINAL = ' '.join(TEXTO_FINAL.split())
            TEXTO_PAGES.append(TEXTO_FINAL)
            #print(regex_text)
            #print('>>>>>>>>>>>>>>>>>')
            #print(TEXTO_FINAL)

        TEXTO_DOCS.append(TEXTO_PAGES)
    return TEXTO_DOCS


In [415]:
Textos_libres = TEXTO_FINAL(PDF_TABLAS, TEXTO_PDFS)